In [1]:
import pandas as pd
import ee
from collections import defaultdict
import numpy as np
import datetime
ee.Initialize()
import random

# Parameters

In [2]:
# GOOGLE DRIVE FOLDER
GDRIVE_FOLDER = 'wfp-allbands-unsup-1'

# 13 bands or RGB?
ALL_BANDS = True



# NUMBER OF FILES TO DOWNLOAD
DOWNLOAD_NB_FILES = 40

# RADIUS AROUND COORD IN METERS
# This is the number of meter around the point coordinate to include in the picture
RADIUS_AROUND = 400


# RANGES FOR BANDS
# Values tested on different location in nepal
# Hypothesis: Should be the same for all images
RANGE_MIN = 0
RANGE_MAX = 2000


# RANGE FOR DATES
# we need to take several pictures to select ones without clouds
# We look for RANDE_DATE weeks around the date
RANDE_DATE = 2

# TIME OF INTEREST
# A random date is selected between TOI_BEGIN and TOI_END
TOI_BEGIN = '01/01/2016'
TOI_END = '01/06/2019'

# MAXIMUM % OF CLOUDS
MAX_CLOUDS = 10

# LAUNCH EXPORT
LAUNCH_EXPORT = True

# REGION TO TAKE PICTURE FROM
NEPAL = ee.Geometry.Polygon([[84.46716739280828,29.108665503908366],
[82.84929902963563,29.95080069391136],
[81.9899776971655,30.413866129203402],
[81.01575580840813,30.491203262843925],
[79.96277286155828,28.849173773303857],
[82.82709294386302,27.405490759555242],
[83.79152604893773,27.280548634789366],
[84.12517363514314,27.354698207754293],
[84.61046405920763,27.23515863534531],
[84.56770354968148,26.99802099337535],
[86.30188418968328,26.48893917833053],
[86.80725528343328,28.128845548855082],
[85.631313693822,28.638952723497773],
[84.46716739280828,29.108665503908366]])

# Google Earth Engine download


In [3]:
# HELPER FUNCTIONS

def stringify_date(date):
    #return f'{date.day}/{date.month}/{date.year}'
    
    return date.strftime("%d/%m/%Y")

def get_name_im(x, y, date):
    return f"{str(x)[:9].replace('.', '-')}_{str(y)[:9].replace('.', '-')}_{date}"

def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())),
    )
def get_date(data_str):
    d, m, y = data_str.split('/')
    
    return datetime.date(int(y), int(m), int(d))

def get_random_coordinate(geometry):
    rd_points = ee.FeatureCollection.randomPoints(geometry,1, random.randint(0, 1e10))
    
    return rd_points.getInfo()['features'][0]['geometry']['coordinates']

# Generate a rectangle containing the circle (centered on the coordinate) with radius RADIUS_AROUND
def get_geometry_radius(geometry_point):
    coord = np.array(geometry_point.getInfo()['coordinates'][0])
    return ee.Geometry.Rectangle([coord[:, 0].min(), coord[:, 1].min(), coord[:, 0].max(), coord[:, 1].max()])

# Generate the dates around the observation date
def date_range_to_collect(input_date, debug=False):
    d, m, y = input_date.split('/')
        
    target_date = datetime.date(int(y), int(m), int(d))
    delta = datetime.timedelta(weeks=RANDE_DATE)
    return target_date-delta, target_date+delta

In [4]:
def generate_images_random(image_collection, debug=True):
  

        start = get_date(TOI_BEGIN)
        end = get_date(TOI_END)
        
        date = random_date(start, end)
        
        x,y = get_random_coordinate(NEPAL)
        
        generate_image(image_collection, x, y, stringify_date(date), get_name_im(x,y,date), debug)


def generate_image(image_collection, x, y, date, image_name, debug=True):
    if debug: print(f'Working on {image_name}: ({x}, {y}) on {date}')
    geo = ee.Geometry.Point(x, y);
    radius = geo.buffer(RADIUS_AROUND);
    geometry_radius= get_geometry_radius(radius)
    
    spatialFiltered = image_collection.filterBounds(geo)

    date_range = date_range_to_collect(date, debug)
    if debug: print('date range:' + str(date_range[0]) + str(date_range[1]))
    temporalFiltered = spatialFiltered.filterDate(str(date_range[0]), str(date_range[1]))


    least_clouds = temporalFiltered.sort('CLOUD_PIXEL_PERCENTAGE').first()



    if ALL_BANDS:
        testimg = least_clouds.select('B.+')
    else:
        testimg = least_clouds.visualize(bands=['B4', 'B3', 'B2'], min=RANGE_MIN, max=RANGE_MAX)
        
    if LAUNCH_EXPORT:
        task = ee.batch.Export.image.toDrive(testimg, folder=GDRIVE_FOLDER, region=geometry_radius.getInfo()['coordinates'][0], description=image_name, scale=10)
        task.start()


In [5]:
dataset = ee.ImageCollection('COPERNICUS/S2').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than',MAX_CLOUDS)

for i, point in enumerate(range(DOWNLOAD_NB_FILES)):
    generate_images_random(dataset)


Working on 85-005396_28-765121_2019-01-25: (85.0053969552374, 28.76512153096905) on 25/01/2019
date range:2019-01-112019-02-08

Working on 83-233625_29-490489_2017-04-12: (83.23362538137575, 29.4904897516145) on 12/04/2017
date range:2017-03-292017-04-26

Working on 81-320737_28-967687_2016-05-24: (81.3207372772009, 28.967687534927936) on 24/05/2016
date range:2016-05-102016-06-07

Working on 84-749490_27-052825_2017-07-03: (84.74949022482048, 27.052825819484337) on 03/07/2017
date range:2017-06-192017-07-17

Working on 83-946094_27-360119_2016-04-01: (83.94609451706732, 27.36011996738946) on 01/04/2016
date range:2016-03-182016-04-15

Working on 83-135956_27-539027_2018-06-17: (83.13595642965633, 27.539027657488933) on 17/06/2018
date range:2018-06-032018-07-01

Working on 82-786748_28-907205_2017-07-22: (82.78674899092245, 28.907205479802258) on 22/07/2017
date range:2017-07-082017-08-05

Working on 85-062190_28-490729_2016-10-17: (85.06219056410055, 28.49072905321274) on 17/10/2016